In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

In [75]:
traffic_raw = []
#Set up the api url and parameters
offset = 155000
increment = 1000
looper = "True"

In [76]:
while looper == "True":
    r = requests.get("https://data.austintexas.gov/resource/dx9v-zd7x.json", params = {
                "$limit": 1000,
                "$offset": offset
                })
    if r.status_code == 200:
        if r.json() == []:
            print("nothing there")
            offset = int(offset - increment)
            increment = int(increment / 2)
            print("Record Start = " + str(offset) + "Pagination is " + str(increment))
        elif increment < 2:
            looper = "False"
        else:        
            traffic_raw.append(r.json())
            print("Record Start = " + str(offset) + "Pagination is " + str(increment))
            offset += increment
        
    else:
        print("***** IT IS BROKEN *****")
        print(r.status_code)

Record Start = 155000Pagination is 1000
Record Start = 156000Pagination is 1000
nothing there
Record Start = 156000Pagination is 500
Record Start = 156000Pagination is 500
Record Start = 156500Pagination is 500
nothing there
Record Start = 156500Pagination is 250
Record Start = 156500Pagination is 250
nothing there
Record Start = 156500Pagination is 125
Record Start = 156500Pagination is 125
nothing there
Record Start = 156500Pagination is 62
Record Start = 156500Pagination is 62
nothing there
Record Start = 156500Pagination is 31
Record Start = 156500Pagination is 31
Record Start = 156531Pagination is 31
nothing there
Record Start = 156531Pagination is 15
Record Start = 156531Pagination is 15
nothing there
Record Start = 156531Pagination is 7
Record Start = 156531Pagination is 7
Record Start = 156538Pagination is 7
nothing there
Record Start = 156538Pagination is 3
Record Start = 156538Pagination is 3
Record Start = 156541Pagination is 3
nothing there
Record Start = 156541Pagination i

In [77]:
pd.DataFrame(traffic_raw)
#Munge the json output that was converted to a data frame
traffic_raw = [{
    "ReportID": rec["traffic_report_id"],
    "PublishedDate": rec["published_date"],
    "IncidentDesc": rec["issue_reported"],
    "Address": rec["address"],
    "Latitude": rec["latitude"],
    "Longitude": rec["longitude"],
    "IncidentDate": rec["traffic_report_status_date_time"],
    "ReportStatus": rec["traffic_report_status"]
} for rec in traffic_raw]
#Convert back to data frame
traffic_incidents = pd.DataFrame(traffic_raw)

TypeError: list indices must be integers or slices, not str

In [71]:
traffic_incidents.shape

(0, 0)

In [79]:
traffic_raw

[[{'traffic_report_id': '773903B72B8835626035A23439016A04591DC73D_1577607134',
   'published_date': '2019-12-29T08:12:14.000Z',
   'issue_reported': 'Crash Urgent',
   'location': '(30.267949,-97.741355)',
   'latitude': '30.267949',
   'longitude': '-97.741355',
   'address': '604 Brazos St',
   'traffic_report_status': 'ARCHIVED',
   'traffic_report_status_date_time': '2019-12-29T10:35:04.000Z'},
  {'traffic_report_id': '8F7A4CF1D8FB52D12619FB0C6CAFC42ABF5A5E61_1577599905',
   'published_date': '2019-12-29T06:11:45.000Z',
   'issue_reported': 'Crash Urgent',
   'location': '(30.314529,-97.76904)',
   'latitude': '30.314529',
   'longitude': '-97.76904',
   'address': '3300 Foothill Dr',
   'traffic_report_status': 'ARCHIVED',
   'traffic_report_status_date_time': '2019-12-29T11:45:04.000Z'},
  {'traffic_report_id': '4A1432C03DFFE532268819E1F8AB3083818ACE50_1577618721',
   'published_date': '2019-12-29T11:25:21.000Z',
   'issue_reported': 'Crash Urgent',
   'location': '(30.222797,-97